# 14 - Streaming 流式输出

## 用途
学习通义千问流式输出技术，支持实时查看生成过程，提升用户体验

## 学习目标
- 理解流式输出原理和优势
- 掌握 stream 方法的使用
- 能实现通义千问基础流式功能
- 掌握流式数据的处理和显示

## 🔑 前置要求
**注意**：需要先完成 LCEL 管道语法学习，理解链式组合原理

## 代码块独立性说明
**注意**：每个代码块都是独立的，包含完整的导入和初始化，确保可以单独运行。

## 1. 流式输出基础概念

In [ ]:
# 流式输出基础概念 - 独立代码块
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
import os
import time

# 加载环境变量
load_dotenv()

print("🌊 流式输出基础概念理解:")
print("=" * 50)

# 检查 OpenAI API 配置
openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    print("❌ 未找到 OPENAI_API_KEY 环境变量")
    print("   请在 .env 文件中添加: OPENAI_API_KEY=your_key_here")
else:
    print(f"✅ OpenAI API Key 已配置 (长度: {len(openai_api_key)})")
    
    print(f"\n📝 流式输出核心概念:")
    print(f"   1. 实时生成：逐个 token 返回结果")
    print(f"   2. 用户体验：减少等待时间")
    print(f"   3. 内存效率：不需要等待完整响应")
    print(f"   4. 迭代器：返回可迭代的流对象")
    
    # 创建基础 LLM
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0.7,
        max_tokens=50
    )
    
    print(f"\n🏗️  创建的组件:")
    print(f"   1. LLM: {type(llm).__name__}")
    print(f"   2. 流式支持: {hasattr(llm, 'stream')}")
    
    # 对比 invoke 和 stream
    print(f"\n🔍 invoke vs stream 对比:")
    
    test_prompt = "请简单介绍一下人工智能"
    
    # 1. 传统 invoke 方法
    print(f"\n   1. 传统 invoke 方法:")
    start_time = time.time()
    invoke_result = llm.invoke(test_prompt)
    end_time = time.time()
    
    print(f"      执行时间: {(end_time - start_time)*1000:.1f} ms")
    print(f"      结果长度: {len(invoke_result.content)} 字符")
    print(f"      完整内容: {invoke_result.content}")
    
    # 2. 流式 stream 方法
    print(f"\n   2. 流式 stream 方法:")
    start_time = time.time()
    stream_chunks = []
    chunk_count = 0
    
    for chunk in llm.stream(test_prompt):
        stream_chunks.append(chunk)
        chunk_count += 1
        print(f"      Token {chunk_count}: {chunk.content}", end="")
        time.sleep(0.1)  # 模拟显示延迟
    
    end_time = time.time()
    print(f"\n      流式执行时间: {(end_time - start_time)*1000:.1f} ms")
    print(f"      Token 数量: {chunk_count}")
    print(f"      合并内容: {''.join(chunk.content for chunk in stream_chunks)}")
    
    # 验证结果一致性
    stream_content = ''.join(chunk.content for chunk in stream_chunks)
    content_match = stream_content == invoke_result.content
    
    print(f"\n📊 结果对比:")
    print(f"   内容一致性: {content_match}")
    print(f"   invoke 长度: {len(invoke_result.content)}")
    print(f"   stream 长度: {len(stream_content)}")
    
    # 验证点：流式输出基础概念正确
    assert chunk_count > 0, "流式输出没有产生 token"
    assert len(stream_content) > 0, "流式输出内容为空"
    assert content_match, "流式和传统输出内容不一致"
    print(f"\n✅ 验证通过：流式输出基础概念理解正确")

## 2. LCEL 管道流式输出

In [ ]:
# LCEL 管道流式输出 - 独立代码块
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os
import time

# 加载环境变量
load_dotenv()

print("🔗 LCEL 管道流式输出测试:")
print("=" * 50)

openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    print("❌ OpenAI API Key 未配置")
else:
    try:
        # 创建 LCEL 管道组件
        llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.7,
            max_tokens=60
        )
        
        prompt = PromptTemplate(
            input_variables=["topic"],
            template="请详细介绍一下 {topic}，分3个要点说明。"
        )
        
        parser = StrOutputParser()
        
        print(f"📝 创建的组件:")
        print(f"   1. Prompt: {type(prompt).__name__}")
        print(f"   2. LLM: {type(llm).__name__}")
        print(f"   3. Parser: {type(parser).__name__}")
        
        # 构建 LCEL 管道
        chain = prompt | llm | parser
        
        print(f"\n🔗 构建 LCEL 管道:")
        print(f"   管道类型: {type(chain)}")
        print(f"   流式支持: {hasattr(chain, 'stream')}")
        
        # 1. 管道 invoke 调用
        print(f"\n🧪 1. 管道 invoke 调用:")
        
        test_topic = "机器学习"
        
        start_time = time.time()
        invoke_result = chain.invoke({"topic": test_topic})
        end_time = time.time()
        
        print(f"   主题: {test_topic}")
        print(f"   执行时间: {(end_time - start_time)*1000:.1f} ms")
        print(f"   结果长度: {len(invoke_result)} 字符")
        print(f"   完整内容: {invoke_result}")
        
        # 2. 管道 stream 调用
        print(f"\n🧪 2. 管道 stream 调用:")
        
        start_time = time.time()
        stream_chunks = []
        chunk_count = 0
        
        print(f"   流式输出开始:")
        for chunk in chain.stream({"topic": test_topic}):
            stream_chunks.append(chunk)
            chunk_count += 1
            print(f"{chunk}", end="", flush=True)
            time.sleep(0.05)  # 模拟实时显示
        
        end_time = time.time()
        print(f"\n\n   流式执行时间: {(end_time - start_time)*1000:.1f} ms")
        print(f"   Token 数量: {chunk_count}")
        
        # 合并流式结果
        stream_content = ''.join(stream_chunks)
        print(f"   合并内容: {stream_content}")
        
        # 3. 不同主题的流式测试
        print(f"\n🧪 3. 不同主题的流式测试:")
        
        topics = ["深度学习", "自然语言处理", "计算机视觉"]
        
        for i, topic in enumerate(topics, 1):
            print(f"\n   主题 {i}: {topic}")
            print(f"   流式输出: ", end="")
            
            topic_chunks = []
            for chunk in chain.stream({"topic": topic}):
                topic_chunks.append(chunk)
                print(f"{chunk}", end="", flush=True)
            
            topic_content = ''.join(topic_chunks)
            print(f"\n   输出长度: {len(topic_content)} 字符")
        
        # 验证点：LCEL 管道流式输出正确
        assert chunk_count > 0, "管道流式输出没有产生 token"
        assert len(stream_content) > 0, "管道流式输出内容为空"
        assert stream_content == invoke_result, "管道流式和传统输出不一致"
        
        print(f"\n✅ 验证通过：LCEL 管道流式输出正确")
        
    except Exception as e:
        print(f"\n❌ LCEL 管道流式输出失败: {e}")

## 3. 流式数据处理和显示

In [ ]:
# 流式数据处理和显示 - 独立代码块
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os
import time
import json

# 加载环境变量
load_dotenv()

print("📊 流式数据处理和显示测试:")
print("=" * 50)

openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    print("❌ OpenAI API Key 未配置")
else:
    try:
        # 创建基础管道
        llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.7,
            max_tokens=80
        )
        
        prompt = PromptTemplate(
            input_variables=["item", "count"],
            template="请列出 {count} 个关于 {item} 的有趣事实。"
        )
        
        chain = prompt | llm | StrOutputParser()
        
        print(f"📝 创建的管道:")
        print(f"   模板: {prompt.template}")
        print(f"   变量: {prompt.input_variables}")
        
        # 1. 基础流式显示
        print(f"\n🧪 1. 基础流式显示:")
        
        def basic_stream_display(chain, inputs, description=""):
            """基础流式显示函数"""
            print(f"   {description}流式输出: ", end="")
            
            chunks = []
            start_time = time.time()
            
            for chunk in chain.stream(inputs):
                chunks.append(chunk)
                print(f"{chunk}", end="", flush=True)
                time.sleep(0.03)
            
            end_time = time.time()
            full_content = ''.join(chunks)
            
            print(f"\n   执行时间: {(end_time - start_time)*1000:.1f} ms")
            print(f"   Token 数量: {len(chunks)}")
            print(f"   内容长度: {len(full_content)} 字符")
            
            return chunks, full_content
        
        # 测试基础显示
        chunks1, content1 = basic_stream_display(
            chain, 
            {"item": "太空探索", "count": 3},
            "太空探索"
        )
        
        # 2. 带统计信息的流式显示
        print(f"\n🧪 2. 带统计信息的流式显示:")
        
        def enhanced_stream_display(chain, inputs, description=""):
            """增强流式显示函数"""
            print(f"   {description}增强流式输出:")
            
            chunks = []
            total_chars = 0
            start_time = time.time()
            
            print(f"   开始输出: ", end="")
            for i, chunk in enumerate(chain.stream(inputs), 1):
                chunks.append(chunk)
                total_chars += len(chunk)
                
                # 显示进度
                print(f"{chunk}", end="", flush=True)
                
                # 每10个token显示统计
                if i % 10 == 0:
                    elapsed = time.time() - start_time
                    rate = total_chars / elapsed if elapsed > 0 else 0
                    print(f" [Token:{i}, 字符:{total_chars}, 速度:{rate:.1f}字符/s]", end="")
                
                time.sleep(0.02)
            
            end_time = time.time()
            full_content = ''.join(chunks)
            
            print(f"\n   最终统计:")
            print(f"     总时间: {(end_time - start_time)*1000:.1f} ms")
            print(f"     总Token: {len(chunks)}")
            print(f"     总字符: {total_chars}")
            print(f"     平均速度: {total_chars/(end_time - start_time):.1f} 字符/s")
            
            return chunks, full_content
        
        # 测试增强显示
        chunks2, content2 = enhanced_stream_display(
            chain,
            {"item": "海洋生物", "count": 4},
            "海洋生物"
        )
        
        # 3. 结构化流式数据处理
        print(f"\n🧪 3. 结构化流式数据处理:")
        
        def structured_stream_processor(chain, inputs, description=""):
            """结构化流式数据处理"""
            print(f"   {description}结构化处理:")
            
            stream_data = {
                "chunks": [],
                "timestamps": [],
                "chunk_sizes": [],
                "cumulative_length": []
            }
            
            start_time = time.time()
            cumulative_length = 0
            
            print(f"   流式输出: ", end="")
            for chunk in chain.stream(inputs):
                current_time = time.time() - start_time
                chunk_size = len(chunk)
                cumulative_length += chunk_size
                
                stream_data["chunks"].append(chunk)
                stream_data["timestamps"].append(current_time)
                stream_data["chunk_sizes"].append(chunk_size)
                stream_data["cumulative_length"].append(cumulative_length)
                
                print(f"{chunk}", end="", flush=True)
                time.sleep(0.03)
            
            end_time = time.time()
            
            # 显示统计信息
            print(f"\n   结构化统计:")
            print(f"     总执行时间: {(end_time - start_time)*1000:.1f} ms")
            print(f"     总Chunk数: {len(stream_data['chunks'])}")
            print(f"     平均Chunk大小: {sum(stream_data['chunk_sizes'])/len(stream_data['chunk_sizes']):.1f}")
            print(f"     最大Chunk: {max(stream_data['chunk_sizes'])}")
            print(f"     最小Chunk: {min(stream_data['chunk_sizes'])}")
            
            return stream_data
        
        # 测试结构化处理
        structured_data = structured_stream_processor(
            chain,
            {"item": "量子计算", "count": 2},
            "量子计算"
        )
        
        # 验证点：流式数据处理和显示正确
        assert len(chunks1) > 0, "基础流式显示失败"
        assert len(chunks2) > 0, "增强流式显示失败"
        assert len(structured_data["chunks"]) > 0, "结构化流式处理失败"
        assert len(content1) > 0, "基础流式内容为空"
        assert len(content2) > 0, "增强流式内容为空"
        
        print(f"\n✅ 验证通过：流式数据处理和显示正确")
        
    except Exception as e:
        print(f"\n❌ 流式数据处理失败: {e}")

## 4. 流式输出性能优化

In [ ]:
# 流式输出性能优化 - 独立代码块
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os
import time
import threading
from concurrent.futures import ThreadPoolExecutor

# 加载环境变量
load_dotenv()

print("⚡ 流式输出性能优化测试:")
print("=" * 50)

openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    print("❌ OpenAI API Key 未配置")
else:
    try:
        # 创建不同配置的 LLM
        llm_fast = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.7,
            max_tokens=30,
            streaming=True
        )
        
        llm_slow = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.7,
            max_tokens=80,
            streaming=True
        )
        
        prompt = PromptTemplate(
            input_variables=["topic"],
            template="请简单介绍 {topic}。"
        )
        
        # 创建不同长度的管道
        fast_chain = prompt | llm_fast | StrOutputParser()
        slow_chain = prompt | llm_slow | StrOutputParser()
        
        print(f"📝 创建的性能测试组件:")
        print(f"   1. 快速管道: max_tokens=30")
        print(f"   2. 慢速管道: max_tokens=80")
        
        # 1. 基础性能对比
        print(f"\n🧪 1. 基础性能对比:")
        
        def measure_stream_performance(chain, inputs, description=""):
            """测量流式性能"""
            print(f"   {description}性能测试:")
            
            start_time = time.time()
            chunks = []
            first_token_time = None
            
            for chunk in chain.stream(inputs):
                if first_token_time is None:
                    first_token_time = time.time()
                chunks.append(chunk)
            
            end_time = time.time()
            
            total_time = end_time - start_time
            first_token_latency = first_token_time - start_time if first_token_time else 0
            
            print(f"     总时间: {total_time*1000:.1f} ms")
            print(f"     首Token延迟: {first_token_latency*1000:.1f} ms")
            print(f"     Token数量: {len(chunks)}")
            print(f"     平均Token间隔: {(total_time/len(chunks))*1000:.1f} ms" if chunks else "无Token")
            print(f"     吞吐量: {len(chunks)/total_time:.1f} Token/s" if total_time > 0 else "无数据")
            
            return {
                "total_time": total_time,
                "first_token_latency": first_token_latency,
                "chunk_count": len(chunks),
                "throughput": len(chunks)/total_time if total_time > 0 else 0
            }
        
        # 性能测试
        test_topic = "人工智能"
        
        fast_stats = measure_stream_performance(
            fast_chain, 
            {"topic": test_topic}, 
            "快速管道"
        )
        
        slow_stats = measure_stream_performance(
            slow_chain, 
            {"topic": test_topic}, 
            "慢速管道"
        )
        
        # 2. 并发流式处理
        print(f"\n🧪 2. 并发流式处理:")
        
        def concurrent_stream_test(chain, inputs_list, max_workers=3):
            """并发流式测试"""
            print(f"   并发测试 (最大并发: {max_workers}):")
            
            def single_stream_task(inputs):
                start_time = time.time()
                chunks = []
                for chunk in chain.stream(inputs):
                    chunks.append(chunk)
                end_time = time.time()
                return {
                    "chunks": chunks,
                    "time": end_time - start_time,
                    "count": len(chunks)
                }
            
            # 串行执行
            start_time = time.time()
            serial_results = []
            for inputs in inputs_list:
                result = single_stream_task(inputs)
                serial_results.append(result)
            serial_time = time.time() - start_time
            
            # 并发执行
            start_time = time.time()
            with ThreadPoolExecutor(max_workers=max_workers) as executor:
                concurrent_results = list(executor.map(single_stream_task, inputs_list))
            concurrent_time = time.time() - start_time
            
            print(f"     串行时间: {serial_time*1000:.1f} ms")
            print(f"     并发时间: {concurrent_time*1000:.1f} ms")
            print(f"     性能提升: {serial_time/concurrent_time:.1f}x")
            
            return serial_results, concurrent_results
        
        # 并发测试
        topics = [
            {"topic": "机器学习"},
            {"topic": "深度学习"},
            {"topic": "自然语言处理"}
        ]
        
        serial_results, concurrent_results = concurrent_stream_test(
            fast_chain, topics, max_workers=3
        )
        
        # 3. 内存使用优化
        print(f"\n🧪 3. 内存使用优化:")
        
        def memory_efficient_stream(chain, inputs, max_buffer_size=10):
            """内存高效流式处理"""
            print(f"   内存高效流式处理 (缓冲区大小: {max_buffer_size}):")
            
            buffer = []
            processed_count = 0
            
            print(f"   流式输出: ", end="")
            for chunk in chain.stream(inputs):
                buffer.append(chunk)
                
                # 当缓冲区满时，处理并清空
                if len(buffer) >= max_buffer_size:
                    # 模拟处理缓冲区内容
                    buffered_content = ''.join(buffer)
                    print(f"{buffered_content}", end="", flush=True)
                    processed_count += len(buffer)
                    buffer.clear()  # 清空缓冲区
                
                time.sleep(0.02)
            
            # 处理剩余内容
            if buffer:
                remaining_content = ''.join(buffer)
                print(f"{remaining_content}", end="", flush=True)
                processed_count += len(buffer)
            
            print(f"\n   处理的Token数量: {processed_count}")
            print(f"   最大内存使用: {max_buffer_size} 个Token")
            
            return processed_count
        
        # 内存优化测试
        memory_result = memory_efficient_stream(
            slow_chain,
            {"topic": "数据科学"},
            max_buffer_size=5
        )
        
        # 4. 性能优化建议
        print(f"\n💡 流式输出性能优化建议:")
        print(f"   1. 控制max_tokens避免过长输出")
        print(f"   2. 使用缓冲区管理内存使用")
        print(f"   3. 并发处理提高吞吐量")
        print(f"   4. 监控首Token延迟和吞吐量")
        print(f"   5. 合理设置temperature平衡速度和质量")
        
        # 验证点：流式输出性能优化正确
        assert fast_stats["chunk_count"] > 0, "快速管道无输出"
        assert slow_stats["chunk_count"] > 0, "慢速管道无输出"
        assert fast_stats["throughput"] > 0, "快速管道吞吐量为0"
        assert slow_stats["throughput"] > 0, "慢速管道吞吐量为0"
        assert memory_result > 0, "内存优化处理失败"
        
        print(f"\n✅ 验证通过：流式输出性能优化正确")
        
    except Exception as e:
        print(f"\n❌ 流式输出性能优化失败: {e}")

## 5. 学习总结与最佳实践

In [ ]:
# 学习总结与最佳实践 - 独立代码块
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

print("📋 Streaming 流式输出学习总结:")
print("=" * 50)

# 验证点检查
verification_points = [
    "✅ 流式原理：逐个token实时输出",
    "✅ stream方法：基础流式调用",
    "✅ LCEL流式：管道流式组合",
    "✅ 数据处理：流式数据收集和显示",
]

for point in verification_points:
    print(point)

print(f"\n🎯 核心技能掌握情况: {len(verification_points)}/4 项")

print("\n💡 Streaming 最佳实践:")
print("1. 实时体验：使用stream减少用户等待")
print("2. 缓冲管理：合理设置缓冲区大小")
print("3. 性能监控：关注首Token延迟和吞吐量")
print("4. 错误处理：流式过程中的异常捕获")
print("5. 内存优化：避免大量数据在内存中累积")

print("\n🔧 流式输出应用场景:")
print("1. 聊天应用：实时显示对话内容")
print("2. 代码生成：逐行显示生成代码")
print("3. 文本创作：实时显示创作过程")
print("4. 数据分析：逐步显示分析结果")
print("5. 长文本处理：避免长时间等待")

print("\n🚀 下一步学习建议:")
print("1. 深入学习 ChatMessageHistory 消息历史")
print("2. 掌握 RunnablePassthrough 数据透传")
print("3. 学习 OutputParser 高级解析技术")
print("4. 探索异步流式处理模式")
print("5. 实践生产级流式应用开发")

# 最终验证：确保 Streaming 基础功能可用
try:
    openai_api_key = os.getenv('OPENAI_API_KEY')
    
    if openai_api_key:
        # 简单流式测试
        llm = ChatOpenAI(model="gpt-4o-mini", max_tokens=20)
        prompt = PromptTemplate(
            input_variables=["text"],
            template="总结: {text}"
        )
        
        chain = prompt | llm | StrOutputParser()
        
        # 测试流式输出
        chunks = []
        for chunk in chain.stream({"text": "流式输出测试"}):
            chunks.append(chunk)
        
        full_content = ''.join(chunks)
        
        print(f"\n🎉 最终验证成功:")
        print(f"   API 状态: 可用")
        print(f"   流式Token数: {len(chunks)}")
        print(f"   输出长度: {len(full_content)} 字符")
        print(f"   输出内容: {full_content}")
        print("\n✅ Streaming 流式输出学习完成！")
        
        print(f"\n🎊 LCEL 编排层技术掌握！")
        print(f"   已掌握: Runnable + RunnableMap + LCEL + Streaming")
        print(f"   下一步: 学习消息历史和数据透传技术")
        
    else:
        print(f"\n⚠️  最终验证跳过: OpenAI API Key 未配置")
        print("   请配置 OPENAI_API_KEY 后重试")
        
except Exception as e:
    print(f"\n❌ 最终验证失败: {e}")